<h1><center>A complete guide on A/B Testing</center></h1>

<h2> Introduction to AB Testing </h2>

A/B testing is a general methodology used online when we want to test new products or features. A/B test allows us to determine scientifically how to optimize products or features (a website or an app or any system) by trying out possible changes and see what performs better with your users. It allows to make data-driven decisions rather than relying on intution about user preferences. To keep the discussion concise, hereafter, we will explain A/B testing as a method employed to determine if a new feature F2 should replace existing feature F1. This feature could be a small change (e.g. changing the font or color of a button on your website) or a complicated change (using deep learning for product recommendation in amazon). This feature could not even be user visible (changing the ranking system of friend suggestion in facebook).  

Broadly speaking, A/B testing uses two sets of users. First set, the control group, is shown Feature F1 and the second set, treatment group/ experiment group is shown the new feature F2. A/B testing uses rigorous post experiment analysis to  to determine which feature performs better with users. A/B testing is generally used when we compare two options. If more than 2 options (of features) are tested, it is called multi-variate A/B testing (or split test). A brief explanation of A/B testing is shown in Figure 1 where a company wants to increase its user engagement.
<img src="figures/Vcs.jpg" width="500">

<h3> Why A/B tests </h3>

<h3> History of A/B testing</h3>

<h3> When it is useful/ when it is not</h3>